# 学習データセットのカタログ検討

In [1]:
from __future__ import unicode_literals
from __future__ import print_function
import sys
import os
import math
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import json
import jsonschema

# インラインでグラフを表示
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (9, 6)

In [2]:
sys.version_info

sys.version_info(major=2, minor=7, micro=13, releaselevel='final', serial=0)

In [3]:
cv2.__version__

'3.1.0'

## カタログの作成

In [4]:
root_dir = os.path.join('..', '..', 'contest', 'APC', 'Single')
camera_image_dir = os.path.join(root_dir, 'Single')
mask_image_dir = os.path.join(root_dir, 'mask_label', 'single')

In [5]:
path = os.path.join(mask_image_dir, '..', 'item_table.csv')
df_items = pd.read_csv(path, encoding='cp932')
df_items.tail(3)

,class,r,g,b
22,23,255,128,255
23,24,0,255,255
24,25,128,255,255


In [6]:
def extract_bounding_boxes(path, df_items):
    boxes = []

    # ラベルの矩形領域と重心位置を取得
    # XXX:このロジックは複数ラベルに対応できない
    labels_image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    width, height, _ = labels_image.shape
    labels_gray = cv2.cvtColor(labels_image, cv2.COLOR_BGR2GRAY)
    #_, labels_binary = cv2.threshold(labels_gray, 0, 255, cv2.THRESH_BINARY)
    #labels_binary = cv2.bitwise_not(labels_binary)
    _, _, stats, centroids = cv2.connectedComponentsWithStats(labels_gray)

    for stat, centroid in zip(stats, centroids):
        b_x, b_y, b_w, b_h, _ = stat[0], stat[1], stat[2], stat[3], stat[4]
        c_x, c_y = int(math.floor(centroid[1])), int(math.floor(centroid[0]))
        # 面積が極端に小さい・大きい領域は誤検出のため除外
        if b_w * b_h <= (width * 2) or b_w * b_h >= (width * height  * 0.95):
            continue
        # 重心位置の画素からクラス値を決定
        r, g, b = labels_image[c_x][c_y]
        clazz = int(df_items[(df_items['r'] == r) & (df_items['g'] == g) & (df_items['b'] == b)]['class'].values[0])
        # 結果を格納
        boxes.append({
            'class': str(clazz), 'x': str(b_x), 'y': str(b_y), 'width': str(b_w), 'height': str(b_h)
        })
    return boxes

In [7]:
def find_label_images(dir_path):
    for root, dirs, files in os.walk(dir_path):
        for f in files:
            if f == str('label.bmp'):
                yield os.path.join(root, f)

In [8]:
dataset = []

for path in find_label_images(mask_image_dir):
    _, clazz, pattern_id, _ = path.split(mask_image_dir)[1].split('/')
    classes = clazz.split('_')
    color_image_path = os.path.join(camera_image_dir, clazz, pattern_id, 'color.bmp')
    depth_image_path = os.path.join(camera_image_dir, clazz, pattern_id, 'depth.bmp.bmp')
    label_image_path = path
    bounding_boxes = extract_bounding_boxes(label_image_path, df_items)
    dataset.append({
        'classes': classes,
        'pattern_id': pattern_id,
        'color_image_path': os.path.abspath(color_image_path),
        'depth_image_path': os.path.abspath(depth_image_path),
        'label_image_path': os.path.abspath(label_image_path),
        'bounding_boxes': bounding_boxes
    })

In [9]:
with open('train_dataset_catalog_sample.json', 'w') as fp:
    json.dump({'dataset': dataset}, fp, sort_keys=True, ensure_ascii=False, indent=2)

## カタログスキーマそのもののチェック

In [10]:
path = os.path.join('..', '..', 'schema', 'train_dataset_catalog_schema.json')
with open(path, 'r') as fp:
    catalog_schema = json.load(fp)
jsonschema.Draft4Validator.check_schema(catalog_schema)

## カタログ情報のスキーマによる検定

In [11]:
with open('train_dataset_catalog_sample.json', 'r') as fp:
    catalog = json.load(fp)
jsonschema.validate(catalog, catalog_schema)